In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

In [14]:
from tensorflow.keras.regularizers import l2

In [7]:
trainpath = '/content/genLogoOutput'
testpath = '/content/output'

In [15]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    shear_range=0.2,
    rotation_range=30,  # Add rotation for more data augmentation
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
train = train_datagen.flow_from_directory(trainpath,target_size =(224,224),batch_size = 16)
test = test_datagen.flow_from_directory(testpath,target_size =(224,224),batch_size = 16)

Found 550 images belonging to 63 classes.
Found 275 images belonging to 63 classes.


In [19]:
vgg = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in vgg.layers:
  layer.trainable = False


In [20]:
from tensorflow.keras.layers import Dropout

In [21]:
# Add new layers with dropout and L2 regularization
x = Flatten()(vgg.output)
x = Dropout(0.5)(x)  # Adding dropout with a rate of 0.5
op = Dense(63, activation='softmax', kernel_regularizer=l2(0.01))(x)  # Adding L2 regularization to the dense layer
vgg16 = Model(vgg.input, op)

In [22]:
vgg19.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [23]:
vgg19.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics =['accuracy'])

In [25]:
vgg19.fit(train,validation_data=test,epochs=20,steps_per_epoch=len(train),validation_steps =len(test))

Epoch 1/20
35/35 [==============================] - 8s 236ms/step - loss: 0.7242 - accuracy: 0.8091 - val_loss: 0.2182 - val_accuracy: 0.9418
Epoch 2/20
35/35 [==============================] - 9s 266ms/step - loss: 0.4618 - accuracy: 0.8745 - val_loss: 0.1238 - val_accuracy: 0.9636
Epoch 3/20
35/35 [==============================] - 9s 270ms/step - loss: 0.4733 - accuracy: 0.8782 - val_loss: 0.1443 - val_accuracy: 0.9564
Epoch 4/20
35/35 [==============================] - 9s 266ms/step - loss: 0.5275 - accuracy: 0.8618 - val_loss: 0.1007 - val_accuracy: 0.9745
Epoch 5/20
35/35 [==============================] - 9s 272ms/step - loss: 0.4143 - accuracy: 0.8855 - val_loss: 0.1534 - val_accuracy: 0.9564
Epoch 6/20
35/35 [==============================] - 9s 251ms/step - loss: 0.4899 - accuracy: 0.8527 - val_loss: 0.0560 - val_accuracy: 0.9855
Epoch 7/20
35/35 [==============================] - 9s 240ms/step - loss: 0.4434 - accuracy: 0.8745 - val_loss: 0.0661 - val_accuracy: 0.9673
Epoch 

In [26]:
vgg19.save('fakelogo.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
